In [1]:
# import torch
import tensorflow as tf

# if torch.cuda.is_available():
#     device = torch.device('cuda')
#     print('Using GPU')
# else:
#     device = torch.device('cpu')
#     print('Using CPU')

In [2]:
# tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [6]:
import time
matrix_size = 32*512

x = torch.randn(matrix_size, matrix_size)
y = torch.randn(matrix_size, matrix_size)

print("using CPU")
start = time.time()
result = torch.matmul(x, y)
print(time.time() - start, "verify device:", result.device)

x_gpu = x.to(device)
y_gpu = y.to(device)
torch.cuda.synchronize()

for i in range(3):
    print("using GPU")
    start = time.time()
    result_gpu = torch.matmul(x_gpu, y_gpu)
    torch.cuda.synchronize()
    print(time.time() - start, "verify device:", result_gpu.device)

using CPU
11.900545120239258 verify device: cpu
using GPU
1.1484346389770508 verify device: cuda:0
using GPU
0.4637339115142822 verify device: cuda:0
using GPU
0.43489646911621094 verify device: cuda:0


In [12]:
# import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json

In [11]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [13]:
x_train = np.load("data/x_train.npy")
y_train = np.load("data/y_train.npy")
x_test = np.load("data/x_test.npy")
y_test = np.load("data/y_test.npy")

In [14]:
input_shape = x_train.shape[1:]
# shape1 = shape[0]
# shape2 = shape[1]
input_shape

(292, 608, 3)

In [15]:
# model Alizadeh and Fazel
from keras.layers import Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Dense, Flatten
from keras.models import Sequential

model_fe = Sequential()

model_fe.add(Conv2D(64, 3, padding='same', input_shape=input_shape))
model_fe.add(BatchNormalization())
model_fe.add(Activation('relu'))
model_fe.add(MaxPooling2D(pool_size=2, strides=None, padding='same'))
model_fe.add(Dropout(0.25))

model_fe.add(Conv2D(128, 3, padding='same'))
model_fe.add(BatchNormalization())
model_fe.add(Activation('relu'))
model_fe.add(MaxPooling2D(pool_size=2, strides=None, padding='same'))
model_fe.add(Dropout(0.25))

model_fe.add(Conv2D(256, 3, padding='same'))
model_fe.add(BatchNormalization())
model_fe.add(Activation('relu'))
model_fe.add(MaxPooling2D(pool_size=2, strides=None, padding='same'))
model_fe.add(Dropout(0.25))

model_fe.add(Conv2D(512, 3, padding='same'))
model_fe.add(BatchNormalization())
model_fe.add(Activation('relu'))
model_fe.add(MaxPooling2D(pool_size=2, strides=None, padding='same'))
model_fe.add(Dropout(0.25))

model_fe.add(Flatten())

model_fe.add(Dense(512))
model_fe.add(BatchNormalization())
model_fe.add(Activation('relu'))
model_fe.add(Dropout(0.25))

model_fe.add(Dense(256))
model_fe.add(BatchNormalization())
model_fe.add(Activation('relu'))
model_fe.add(Dropout(0.25))

model_fe.add(Dense(5))
model_fe.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
model_fe.compile(
    loss='SparseCategoricalCrossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

hist_fe = model_fe.fit(
    x_train, 
    y_train,
    epochs=50,
    batch_size=30,
    validation_data=(x_test, y_test)
    )

AttributeError: 'SparseCategoricalCrossentropy' object has no attribute '__name__'

In [9]:
import keras
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 0.6 sometimes works better for folks
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))